In [3]:
import sqlalchemy as db
import os
import pandas as pd

# Get mySQL password from environment variable
myPassword = os.environ['mySQLPassword']
dbURL="mysql+mysqlconnector://root:{}@127.0.0.1:3306/publications".format(myPassword)
engine = db.create_engine(dbURL)

# STEP 1: Calculate the royaltie of each sales for each author

In [4]:
# Step 1: Join all of the necessary columns into 1

query = '''SELECT sales.title_id, titleauthor.au_id, sales.qty, titles.price, titles.royalty, titleauthor.royaltyper
           FROM sales
           LEFT JOIN titles ON sales.title_id = titles.title_id
           LEFT JOIN titleauthor ON sales.title_id = titleauthor.title_id
           '''

In [5]:
pd.read_sql(query, engine)

,title_id,au_id,qty,price,royalty,royaltyper
0,BU1032,213-46-8915,5,19.99,10,40
1,BU1032,409-56-7008,5,19.99,10,60
2,PS2091,899-46-2035,3,10.95,12,50
3,PS2091,998-72-3567,3,10.95,12,50
4,PC8888,427-17-2319,50,20.00,10,50
5,PC8888,846-92-7186,50,20.00,10,50
6,PS2091,899-46-2035,75,10.95,12,50
7,PS2091,998-72-3567,75,10.95,12,50
8,PS2091,899-46-2035,10,10.95,12,50
9,PS2091,998-72-3567,10,10.95,12,50


In [6]:
# Step 1b: Calculate the royalities by title 

query = '''SELECT ONE.title_id, ONE.au_id, (ONE.price*ONE.qty)*(ONE.roy/100)*(ONE.per/100) as 'royalty'
               FROM (SELECT sales.title_id, titleauthor.au_id, sales.qty, titles.price, titles.royalty as 'roy', titleauthor.royaltyper as 'per'
               FROM sales
               LEFT JOIN titles ON sales.title_id = titles.title_id
               LEFT JOIN titleauthor ON sales.title_id = titleauthor.title_id) AS ONE
           '''

# See the formula to calculate royalties per title per author
# sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100


In [7]:
pd.read_sql(query, engine)

,title_id,au_id,royalty
0,BU1032,213-46-8915,3.998
1,BU1032,409-56-7008,5.997
2,PS2091,899-46-2035,1.971
3,PS2091,998-72-3567,1.971
4,PC8888,427-17-2319,50.000
5,PC8888,846-92-7186,50.000
6,PS2091,899-46-2035,49.275
7,PS2091,998-72-3567,49.275
8,PS2091,899-46-2035,6.570
9,PS2091,998-72-3567,6.570


# Step 2: Aggregate the total royalties for each title for each author

In [30]:
query = '''SELECT title_id, au_id, sum(royalty) as 'royal'
            FROM (SELECT ONE.title_id, ONE.au_id, (ONE.price*ONE.qty)*(ONE.roy/100)*(ONE.per/100) as 'royalty'
               FROM (SELECT sales.title_id, titleauthor.au_id, sales.qty, titles.price, titles.royalty as 'roy', titleauthor.royaltyper as 'per'
               FROM sales
               LEFT JOIN titles ON sales.title_id = titles.title_id
               LEFT JOIN titleauthor ON sales.title_id = titleauthor.title_id) AS ONE) AS TWO
               GROUP BY title_id, au_id
               ORDER BY royal DESC
           '''

In [31]:
pd.read_sql(query, engine)

,title_id,au_id,royal
0,PC1035,238-95-7766,110.160
1,TC3218,807-91-6654,83.800
2,PS2091,899-46-2035,70.956
3,PS2091,998-72-3567,70.956
4,PC8888,427-17-2319,50.000
5,PC8888,846-92-7186,50.000
6,TC4203,648-92-1872,33.460
7,PS1372,756-30-7391,32.385
8,PS3333,172-32-1176,29.985
9,BU7832,274-80-9391,29.985


# Step 3: Calculate the total profits of each author

In [59]:
# Step 3A get the profit per author per title
query = ''' SELECT au_id, THREE.title_id, (THREE.royalty+titles.advance) as 'profit'
            FROM (SELECT title_id, au_id, sum(royalty) as 'royalty'
            FROM (SELECT ONE.title_id, ONE.au_id, (ONE.price*ONE.qty)*(ONE.roy/100)*(ONE.per/100) as 'royalty'
               FROM (SELECT sales.title_id, titleauthor.au_id, sales.qty, titles.price, titles.royalty as 'roy', titleauthor.royaltyper as 'per'
               FROM sales
               LEFT JOIN titles ON sales.title_id = titles.title_id
               LEFT JOIN titleauthor ON sales.title_id = titleauthor.title_id) AS ONE) AS TWO
               GROUP BY title_id, au_id
               ORDER BY royalty DESC) AS THREE
            LEFT JOIN titles
            ON THREE.title_id = titles.title_id
            ORDER BY profit DESC;
           '''

In [60]:
pd.read_sql(query, engine)

,au_id,title_id,profit
0,722-51-5454,MC3021,15021.528
1,899-46-2035,MC3021,15007.176
2,213-46-8915,BU2075,10150.116
3,427-17-2319,PC8888,8050.000
4,846-92-7186,PC8888,8050.000
5,672-71-3249,TC7777,8011.992
6,267-41-2394,TC7777,8008.994
7,472-27-2349,TC7777,8008.994
8,238-95-7766,PC1035,7110.160
9,807-91-6654,TC3218,7083.800


In [89]:
# Step 3B get the profit per author
query = ''' SELECT FOUR.au_id, sum(FOUR.profit) as 'totalProfit'
            FROM (SELECT au_id, THREE.title_id, (THREE.royalty+titles.advance) as 'profit'
            FROM (SELECT title_id, au_id, sum(royalty) as 'royalty'
            FROM (SELECT ONE.title_id, ONE.au_id, (ONE.price*ONE.qty)*(ONE.roy/100)*(ONE.per/100) as 'royalty'
               FROM (SELECT sales.title_id, titleauthor.au_id, sales.qty, titles.price, titles.royalty as 'roy', titleauthor.royaltyper as 'per'
               FROM sales
               LEFT JOIN titles ON sales.title_id = titles.title_id
               LEFT JOIN titleauthor ON sales.title_id = titleauthor.title_id) AS ONE) AS TWO
               GROUP BY title_id, au_id
               ORDER BY royalty DESC) AS THREE
            LEFT JOIN titles
            ON THREE.title_id = titles.title_id
            ORDER BY profit DESC) AS FOUR
            GROUP BY FOUR.au_id
            ORDER BY totalProfit DESC;
           '''

In [90]:
pd.read_sql(query, engine)

,au_id,totalProfit
0,899-46-2035,17353.132
1,213-46-8915,15162.110
2,722-51-5454,15021.528
3,267-41-2394,13020.944
4,724-80-9391,12028.720
5,998-72-3567,8363.456
6,427-17-2319,8050.000
7,846-92-7186,8050.000
8,672-71-3249,8011.992
9,472-27-2349,8008.994


In [102]:
# Step 3B get the profit per author
query = ''' SELECT FOUR.au_id, sum(FOUR.profit) as 'totalProfit'
            FROM (SELECT au_id, THREE.title_id, (THREE.royalty+titles.advance) as 'profit'
            FROM (SELECT title_id, au_id, sum(royalty) as 'royalty'
            FROM (SELECT ONE.title_id, ONE.au_id, (ONE.price*ONE.qty)*(ONE.roy/100)*(ONE.per/100) as 'royalty'
               FROM (SELECT sales.title_id, titleauthor.au_id, sales.qty, titles.price, titles.royalty as 'roy', titleauthor.royaltyper as 'per'
               FROM sales
               LEFT JOIN titles ON sales.title_id = titles.title_id
               LEFT JOIN titleauthor ON sales.title_id = titleauthor.title_id) AS ONE) AS TWO
               GROUP BY title_id, au_id
               ORDER BY royalty DESC) AS THREE
            LEFT JOIN titles
            ON THREE.title_id = titles.title_id
            ORDER BY profit DESC) AS FOUR
            GROUP BY FOUR.au_id
            ORDER BY totalProfit DESC
            LIMIT 3;
           '''

In [103]:
pd.read_sql(query, engine)

,au_id,totalProfit
0,899-46-2035,17353.132
1,213-46-8915,15162.110
2,722-51-5454,15021.528
